In [2]:
import pandas as pd
import sqlite3 as sql
import numpy as np
import re
import string

# TO-DO:
1) Fix column names in grid mix dataframe

2) Figure out how to account for PHEV emissions (relevant papers?)

3) Figure out how to account for HEV battery, if necessary

4) Figure out how to distinguish NIMH/LI battery data

5) Clean up vehicle_type vs vehicle_class

6) Integrate zip-code choice

7) Integrate into django Model ORM

In [ ]:
## create combined vehicle emissions:

## join curb_weight_join to weighted total c02 emissions by vehicle
## need to get embodied emissions and in-use emissions 
## embodied_emissions = vehicleCo2_vector*vehicle_C02_weightings*mass_vehicle
## in-use emissions = Co2/mile = mpg(e)*carbon_intensity
## carbon intensity = carbon intensity of fuel or carbon intensity of grid_mix 
## grid_mix carbon_intensitty = c02 emissions at entered zip code or take average of all zip codes
## append in-use emissions, embodied emissions, and total emissions (embodied+in_use) to joined vehicle data

In [6]:
## load in data
## need vehicle data, emissions weightings/intensities by vehicle, zip data
vehicle_data = pd.read_csv("/Users/josheverts/MIMS-Capstone/Data/joined_2023_vehicle_data.csv")
subregion_emissions = pd.read_csv("/Users/josheverts/MIMS-Capstone/Data/subregion_emissions.csv")

## embodied emissions data (includes both weightings and intensities in the same table)
LI_emissions_data = pd.read_csv("/Users/josheverts/MIMS-Capstone/Data/emissions_factor_LI.csv")
NIMH_emissions_data = pd.read_csv("/Users/josheverts/MIMS-Capstone/Data/emissions_factor_NIMH.csv")
vehicle_emissions_data = pd.read_csv("/Users/josheverts/MIMS-Capstone/Data/emissions_factor_vehicle.csv")

In [29]:
## first average subregion emissions
## use CO2 because vehicle data doesn't include CO2e (until we calculate it ourselves)
## in this table, values are in lbs/MWh
avg_CO2Rate = subregion_emissions['CO2Rate'].mean()
## convert from lbs/MWh to g/kWh
avg_CO2Rate = avg_CO2Rate*453.592/1000

In [24]:
vehicle_data[vehicle_data.atvType == 'PHEV']

,Unnamed: 0,VehicleID_x,Year_x,MFRCode,Make,Model,Range,RangeHwy,atvType,UHighway,UCity,city08U,highway08U,VehicleID_y,Year_y,CurbWeight
231,231,46245,2023,VGA,bentley,flying spur hybrid,0,0.0,PHEV,30.5000,21.5000,17.2268,21.9137,38,2023.0,5690.0
232,232,46248,2023,BMX,bmw,x5 xdrive45e,0,0.0,PHEV,31.9000,24.3000,19.2381,22.0961,49,2023.0,5646.0
233,233,46249,2023,CRX,chrysler,pacifica hybrid,0,0.0,PHEV,42.7998,42.7429,29.4158,29.6499,79,2023.0,5069.0
234,234,46252,2023,CRX,jeep,wrangler 4dr 4xe,0,0.0,PHEV,29.8000,27.6000,20.3076,19.8362,194,2023.0,5361.0
235,235,46253,2023,CRX,jeep,grand cherokee 4xe,0,0.0,PHEV,37.4000,30.3000,22.8474,24.2990,187,2023.0,5396.0
236,236,46258,2023,PRX,porsche,panamera turbo s e-hybrid,0,0.0,PHEV,30.1000,22.3000,17.8416,21.6794,352,2023.0,5408.0
237,237,46259,2023,PRX,porsche,cayenne turbo s e-hybrid,0,0.0,PHEV,26.9000,21.2000,17.0142,19.5061,346,2023.0,5677.0


In [49]:
vehicle_emissions_data.columns

Index(['Unnamed: 0', 'alph_index', 'Material_x', '     VOC', '     CO',
       '     NOx ', '     PM10', '     PM2.5', '     SOx', '     BC',
       '     OC', '     CH4', '     N2O', '     CO2',
       '     CO2 (VOC, CO, CO2)', '     GHGs', 'Material_y',
       'ICEV: Conventional Material', 'ICEV: Lightweight Material',
       'HEV: Conventional Material', 'HEV: Lightweight Material',
       'PHEV: Conventional Material', 'PHEV: Lightweight Material',
       'EV: Conventional Material', 'EV: Lightweight Material',
       'FCV: Conventional Material', 'FCV: Lightweight Material'],
      dtype='object')

In [60]:
## define functions to import as a file
## Vehicle is planned object which will be one entry taken from EPA vehicle testing database
## User is planned object which will store a user's input
def est_battery_capacity_from_vehicle(Vehicle):
    '''return battery capacity in kwh'''
    kwh_size = Vehicle.RangeHwy*(1/Vehicle.highway08U)*33.7
    return kwh_size

def est_battery_weight_from_vehicle(Vehicle):
    ''' return battery weight in lbs'''
    battery_weight = est_battery_capacity_from_vehicle(Vehicle)*1/(.156)*2.204 ## assume .156 kWh/Kg energy density
    return battery_weight 

## vehicle data (here we assume constant for all vehicle types to be changed later)
# non_battery_mass_fractions = [.599, .153, .082, .046, .039, .024, .019, .009, 0]
# remainder_non_battery = 1-sum(non_battery_mass_fractions)
# non_battery_mass_fractions = non_battery_mass_fractions + [remainder_non_battery]
# non_battery_emissions_vector = [1037, 1358, 1644, 732, 1782, 360, 1949, 5163, 1734]

## vehicle material transformation (includes assembly, disposal, recycling)
## for now assume constant (not depending on vehicle type or weight)
# vehicle_mat_transform = 1.5e6 ## grams/lifetime 

## vehicle battery manufacture (includes both lead acid and li-ion batteries)
## assume constant li-ion term across vehicle types: 60,000 g (3 battery replacements)
## assume Co2 emissions in grams are roughly 4e3 times battery weight (highly simplified model for now)

def weighted_emissions_calc(vehicle_weight, mass_fractions, emissions_vector):
    mass_fractions = np.array(mass_fractions)
    emissions_vector = np.array(emissions_vector)
    return np.sum(vehicle_weight*mass_fractions*emissions_vector)


##
## need to fix this to work with Hybrid vehicles (how to estimate HEV battery size?)
## update to also work with lightweight versions correctly
##
def get_embodied_emissions(vehicle_weight, battery_weight, vehicle_type, emissions_type, vehicle_emissions_manufacturing, battery_emissions_manufacturing):
    vehi_sum = weighted_emissions_calc(vehicle_weight, vehicle_emissions_manufacturing[str(emissions_type)], 
                                       vehicle_emissions_manufacturing[str(vehicle_type)])
    if vehicle_type == 'ICEV: Conventional Material' or 'HEV: Conventional Material':
        battery_sum = 0
    else:
        battery_sum = weighted_emissions_calc(vehicle_weight, battery_emissions_manufacturing[str(emissions_type)], 
                                              battery_emissions_manufacturing[str(vehicle_type)])
        
    ## terms assumed constant:
    lead_acid_battery = 60000
    fluids = 634200
    battery_sum+=lead_acid_battery
    return vehi_sum + battery_sum + fluids

# def est_embodied_emissions(Vehicle):
#     ## account for EV battery or ICE
#     if Vehicle.battery == True:
#         battery = True
#         kwh_size = est_battery_capacity_from_vehicle(Vehicle)
#     else:
#         battery = False
#         kwh_size = 0
#     vehicle_weight = Vehicle.weight
#     ## send estimated battery capacity and vehicle weight to embodied emissions function
#     embodied_emissions = get_embodied_emissions(vehicle_weight, battery_weight)  
#     return embodied_emissions

# def get_grid_carbon(user_zip, Zip, Co2):
#     ## store lookup table of CO2 intensity by grid region
#     ## convert User zipcode input to grid region
# #     user_region = Zip[Zip['ZIP'] == user_zip]
# #     Co2 = Co2[Co2['Co2'] == user_region]
#     return Co2
    

def get_usage_emissions(vehicle_fuel_economy, vehicle_class, miles_estimated, grid_intensity = avg_CO2Rate):

    if vehicle_class == 'ICE' or vehicle_class == 'HEV':
        return vehicle_fuel_economy*miles_estimated*8.9 ## 8.9 kgC02 per gallon
    elif vehicle_class == 'EV':
        grid_carbon = grid_intensity*((vehicle_fuel_economy/33.7)**-1)*miles_estimated ## convert mpge to kwh/mile, return gC
        return grid_carbon ## carbon per kwh*kwh/gallon
    elif vehicle_class == 'PHEV': ## assume vehicle is driven in combined cycle (battery and engine)
        return vehicle_fuel_economy*miles_estimated*8.9 ## 8.9 kgC02 per gallon
    else:
        return 0

# def est_lifetime_emissions(Vehicle, User):
#     ## account for EV battery or ICE
#     embodied_emissions = est_embodied_emisions(Vehicle)
#     usage_emissions = est_usage_emissions(Vehicle,User)
#     return embodied_emissions+usage_emissions

def conv_or_lw(vehicle_class, lightweight=False):
    '''input vehicle_class (Vehicle.atvType) and Boolean (lightweight = False for conventional)'''
    
    possible_columns = ['ICEV: Conventional Material', 'ICEV: Lightweight Material',
       'HEV: Conventional Material', 'HEV: Lightweight Material',
       'PHEV: Conventional Material', 'PHEV: Lightweight Material',
       'EV: Conventional Material', 'EV: Lightweight Material',
       'FCV: Conventional Material', 'FCV: Lightweight Material']
    
    if not lightweight:
        if vehicle_class == 'ICE':
            return possible_columns[0]
        elif vehicle_class == 'HEV':
            return possible_columns[2]
        elif vehicle_class == 'PHEV':
            return possible_columns[4]
        elif vehicle_class == 'EV':
            return possible_columns[6]
        else:
            return possible_columns[8]
    elif lightweight:
        if vehicle_class == 'ICE':
            return possible_columns[1]
        elif vehicle_class == 'HEV':
            return possible_columns[3]
        elif vehicle_class == 'PHEV':
            return possible_columns[5]
        elif vehicle_class == 'EV':
            return possible_columns[7]
        else:
            return possible_columns[9]
    else:
        print("there was an error with get_emissions_weights lightweight parameter")
        return
        
        
def process_vehicle(Vehicle, Manufacturing, Battery, emissions_types):
    vehicle_fuel_economy = Vehicle.highway08U*.55 + Vehicle.city08U*.45 ## combined fuel economy (EPA.gov)
    vehicle_weight = Vehicle.CurbWeight
    vehicle_class = Vehicle.atvType
    vehicle_type = conv_or_lw(vehicle_class)
    battery_weight = est_battery_weight_from_vehicle(Vehicle)
    
    lifetime_emissions = []
    for emittant in emissions_types:
        embodied = get_embodied_emissions(vehicle_weight, battery_weight, 
                                          vehicle_type, emittant, 
                                          Manufacturing, Battery)
        usage = get_usage_emissions(vehicle_fuel_economy, vehicle_class, miles_estimated=178000)
        lifetime_emissions.append([emittant, embodied, usage, (embodied + usage)])
        
    return lifetime_emissions
        
        

In [61]:
## Approach: since here we aren't taking a specific zip code, average C02 emissions across all zip codes
## then for a given vehicle type calculate in-use emissions according to efficiency*carbon_intensity
## for ICE vehicles, use mpge*cabon_intensity_petrol
## for EVs, use average grid mix intensity
## for PHEVs, use mpge*carbon_intensity_petrol, assume combined cycle driving

In [72]:
## for each row of vehicle df (a Vehicle) get emissions for ['CO2']
## assuming lithium batteries for now, because we are ignoring HEV batteries (as they are very small)
## fix column names in vehicle emissionsn data
emissions_array = []
for i in range(len(vehicle_data)):
    Vehicle = vehicle_data.iloc[i]
    lifetime_emissions = process_vehicle(Vehicle, vehicle_emissions_data, LI_emissions_data, ['     CO2'])
    emissions_array.append(lifetime_emissions)
emissions_array = np.array(emissions_array).squeeze()

In [73]:
## take last three columns of emissions_array data and save it to new dataframe for export
vehicle_data['embodied_emissions'], vehicle_data['usage_emissions'], vehicle_data['total_emissions'] = [emissions_array[:,1], emissions_array[:,2], emissions_array[:,3]]

In [75]:
vehicle_data.to_csv("emissions_vehicle_join.csv")